In [2]:
!pip install -q pinecone-client kaggle gdown

In [2]:
import numpy as np
from tqdm.auto import tqdm
from kaggle_secrets import UserSecretsClient
import pickle


# Constants
PINECONE_API_KEY = "b52dac1e-0eb8-47d3-b5ca-ef64ab2dbfcd"
PINECONE_INDEX_NAME = "vn-news-v3"
PINECONE_DIMENSION = 1024
PINECONE_METRIC = "cosine"
PINECONE_ENVIRONMENT = "aws"
PINECONE_REGION = "us-east-1"
BATCH_SIZE = 100


In [15]:
data_path = "src/api/model"
with open(f'{data_path}/total_chunks.pkl', 'rb') as f:
    total_chunks = pickle.load(f)
with open(f'{data_path}/total_output_clean.pkl', 'rb') as f:
    total_output_clean = pickle.load(f)
with open(f'{data_path}/total_text_embeddings.npy', 'rb') as f:
    total_text_embeddings = np.load(f)
with open(f'{data_path}/total_url_chunks.pkl', 'rb') as f:
    total_url_chunks = pickle.load(f)
with open(f'{data_path}/total_urls.pkl', 'rb') as f:
    total_urls = pickle.load(f)
    
index = 0
print(total_chunks[index])
print("===============")
print(total_output_clean[index])
print("===============")
print(total_text_embeddings[index])
print("===============")
print(total_url_chunks[index])
print("===============")
print(total_urls[index])
print("===============")

Tài tử Daniel Day-Lewis tái xuất sau 7 năm vắng bóng 
Daniel Day-Lewis tái xuất màn ảnh rộng với phim "Anemone", do con trai ông đạo diễn.
Dự án đánh dấu sự trở lại của Daniel Day-Lewis sau Phantom Thread - ra mắt năm 2017. Theo Variety, kịch bản do ông và con trai - Ronan Day-Lewis - viết, nội dung xoay quanh tình cảm cha con, tình anh em, các mối quan hệ khác trong gia đình.
Hôm 1/9, chủ tịch Focus Features Peter Kujawski nói: "Chúng tôi không thể vui mừng hơn khi hợp tác Ronan. Cha con Daniel đã viết một kịch bản thực sự xuất sắc, chúng tôi mong đợi đưa góc nhìn của họ đến với khán giả".
Ngoài Daniel Day-Lewis, dàn diễn viên gồm Sean Bean, Samantha Morton, Samuel Bottomley và Safia Oakley-Green. Ben Fordesman làm đạo diễn hình ảnh, Jane Petrie đảm nhận vị trí thiết kế trang phục và Chris Oddy là nhà thiết kế sản xuất.
Ronan Day-Lewis, 26 tuổi, là nhà làm phim và họa sĩ. Anh ra mắt triển lãm nghệ thuật tại Hong Kong ngày 2/10, trước khi tổ chức ở New York và Los Angeles (Mỹ).
Daniel 

In [11]:
print(len(total_url_chunks))
print(len(total_text_embeddings))
print(len(total_output_clean))
print(len(total_chunks))
print(len(total_urls))

694
694
694
694
519


In [14]:
embeddings = np.load('/kaggle/input/llmsss/total_text_embeddings (1).npy')
with open('/kaggle/input/llmsss/total_url_chunks (1).pkl', 'rb') as f:
    urls = pickle.load(f)

assert len(embeddings) == len(urls), "Mismatch between embeddings and URLs"

# Mapping embeddings and URLs
mapped_data = [{"embedding": embeddings[i], "url": urls[i]} for i in range(len(embeddings))]

print("Sample mappings:")
for i in range(5): 
    print(f"Embedding {i}: {mapped_data[i]['embedding'][:5]}...") 
    print(f"URL {i}: {mapped_data[i]['url']}\n")

Sample mappings:
Embedding 0: [-0.00900428 -0.00153056  0.06006427 -0.01960655  0.0191905 ]...
URL 0: https://vnexpress.net/tai-tu-daniel-day-lewis-tai-xuat-sau-7-nam-vang-bong-4799294.html


Embedding 1: [-0.04414823 -0.00360335  0.00814275 -0.02345438  0.02430473]...
URL 1: https://vnexpress.net/ceo-tien-nguyen-du-tuan-thoi-trang-paris-milan-4799155.html


Embedding 2: [-0.01659253  0.01404008  0.02703206 -0.03626209  0.04872503]...
URL 2: https://vnexpress.net/dien-mao-ha-noi-thoi-can-dai-qua-sach-4798290.html


Embedding 3: [-0.00297023  0.00507443  0.03325117 -0.01850397  0.00785883]...
URL 3: https://vnexpress.net/dien-mao-ha-noi-thoi-can-dai-qua-sach-4798290.html


Embedding 4: [-0.01622958 -0.00970322  0.01048953 -0.018901    0.01179618]...
URL 4: https://vnexpress.net/katy-perry-dien-vay-lam-tu-tui-hanh-ly-4798854.html




In [21]:
from pinecone import Pinecone

pc = Pinecone(api_key="b52dac1e-0eb8-47d3-b5ca-ef64ab2dbfcd")
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize Pinecone
# pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# if PINECONE_INDEX_NAME not in pc.list_indexes().names():
#     pc.create_index(
#         name=PINECONE_INDEX_NAME, 
#         dimension=PINECONE_DIMENSION, 
#         metric=PINECONE_METRIC,
#         spec=ServerlessSpec(
#             cloud=PINECONE_ENVIRONMENT,
#             region=PINECONE_REGION
#         )
#     )
#     print(f"Index '{PINECONE_INDEX_NAME}' created.")

# index = pc.Index(PINECONE_INDEX_NAME)

for i in tqdm(range(0, len(mapped_data), BATCH_SIZE)):
    batch_data = mapped_data[i:i + BATCH_SIZE]
    
    to_upsert = [(str(i + j), batch_data[j]["embedding"], {"url": batch_data[j]["url"]}) 
                 for j in range(len(batch_data))]
    
    # Upsert the batch to Pinecone
    index.upsert(vectors=to_upsert)

print(f"Successfully uploaded {len(embeddings)} vectors to the Pinecone index.")


  0%|          | 0/7 [00:00<?, ?it/s]

Successfully uploaded 694 vectors to the Pinecone index.


In [5]:
# import pickle

# with open('/kaggle/input/vn-news/total_url_chunks.pkl', 'rb') as file:
#     total_url_chunks = pickle.load(file)

# with open('/kaggle/input/vn-news-output/total_output_clean.pkl', 'rb') as file:
#     total_output_clean = pickle.load(file)

# mapped_data = list(zip(total_url_chunks, total_output_clean))

# for url_chunk, output_clean in mapped_data[:5]:
#     print(f"URL Chunk: {url_chunk}")
#     print(f"Content: {output_clean}\n")
    
# mapped_data = {key: (total_url_chunks[key], total_output_clean[key]) for key in total_url_chunks if key in total_output_clean}

# for key, (url_chunk, output_clean) in list(mapped_data.items())[:5]:
#     print(f"Key: {key}")
#     print(f"URL Chunk: {url_chunk}")
#     print(f"Content: {output_clean}\n")


URL Chunk: https://vnexpress.net/xu-huong-tuyen-dung-ceo-part-time-4797751.html

Content: Why do you often wake up in the middle of the night: diet, nocturia, age, stress anxiety are all factors that cause many people to wake up in the middle of the night without being able to fall asleep again. Have you ever stared at the empty space at 3: 00 or 4: 00 a. m.? Is that just bad habit or is something more serious going on and why does it always happen at this time? "We start getting shallow sleep around 4 to 5: 00 p. m.," says Lisa Artis, deputy executive director of the charity The Sleep Charity. When people are at this stage, it's easy to wake up. If you normally go to bed around 11: 00 p. m., getting up at 4: 00 a. m. is likely. There are many factors that lead to these unwanted awakenings. "Sleep is regulated by the body's circadian clock, or circadian rhythm. One of the most important circadian rhythms is the sleep-wake cycle," says Artis. Sleep is regulated by two hormones, melatoni